## 적대적 생성 신경망

#### GAN

- 생성자와 판별자의 적대적 학습

- 생성자는 더욱 진짜같은 이미지를 생성하고, 판별자는 더욱 정확히 분류하도록 하여, `경쟁적`으로 발전시키는 구조

- 판별자
  - 실제 이미지를 입력하여 '진짜'로 분류하도록 학습시킴

  - 생성자가 생성한 모조 이미지를 입력하여 '가짜'로 분류하도록 학습시킴
  <br><br>
- 생성자

  - 진짜 이미지와 유사한 모조 이미지를 생성하도록 학습시킴


#### 동작 원리

- 판별자 D : 이미지 `x`가 주어졌을 때, 진짜 이미지일 확률 `D(x)`를 반환함

- 생성자 G : 임의의 노이즈로부터, 모조 이미지 `G(*)`를 생성함

  -> 생성자가 만든 모조 이미지 `G(z)`를 판별자의 입력으로 주면, 모조 이미지가 실제 이미지일 확률 `D(G(z))`를 반환하게 됨

<img src="https://thebook.io/img/080289/710_2.jpg" width="450px">

#### 손실함수

##### - 전체 손실함수

<img src="https://thebook.io/img/080289/fn2-91.jpg" width="450px">

- `x~Pdata(x)` : **실제 데이터**에 대한 확률 분포에서 샘플링한 데이터

- `z~Pz(z)` : **임의의 노이즈**에서 샘플링한 데이터

- `D(x)` : 판별자 D(x)가, 1에 가까우면 진짜 데이터로 판단 / 0에 가까우면 가짜 데이터로 판단

- `D(G(z))` : 생성된 이미지 G(z)가, 1에 가까우면 진짜 데이터로 판단 / 0에 가까우면 가짜 데이터로 판단

- 판별자 D

  - 실제 이미지 `x`를 입력 받을 경우, `D(x)`를 1로 예측함

  - 모조 이미지 `G(z)`를 입력 받을 경우, `D(G(z))`를 0으로 예측함
    
    -> 따라서, 판별자가 모조 이미지를 입력 받았을 때, 1로 예측하도록 하는 것이 목표

##### - 판별자 손실함수

<img src="https://thebook.io/img/080289/fn2-81.jpg" width="200px">

- 최상의 결과 :

  - `D(x) = 1` : 실제 이미지를 '진짜'로 분류
  
  - `D(G(z)) = 0` : 모조 이미지를 '가짜'로 분류
<br><br>
- 위 식을 최대로 하는 방향으로 업데이트하여, **판별 능력**을 키움

##### - 생성자 손실함수

<img src="https://thebook.io/img/080289/fn2-82.jpg" width="120px">

- 최상의 결과 :

  - `D(G(z)) = 1` : 모조 이미지를 '진짜'로 분류
<br><br>
- 위 식을 최소로 하는 방향으로 업데이트하여, 진짜 같은 **이미지 생성 능력**을 키움

---

#### (1) 라이브러리 호출

In [14]:
import os
import numpy as np

import imageio
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt

from torchvision.utils import make_grid, save_image
import torchvision.datasets as datasets
import torchvision.transforms as transforms

#### (2) 변수 설정

In [47]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [49]:
batch_size = 512
epochs = 20
sample_size = 64
nz = 128
k = 1

#### (3) 이미지 데이터 다운로드 : MNIST

In [50]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, )),
])

train_dataset = datasets.MNIST(
    root="/content/data", train=True, transform=transform, download=True
)

train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=4
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


#### (4) 모델 생성

In [51]:
class Generator(nn.Module):
    def __init__(self, nz):
        super(Generator, self).__init__()

        self.nz = nz # 잠재벡터 크기

        self.main = nn.Sequential(
            nn.Linear(self.nz, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 784),
            nn.Tanh()
        )


    def forward(self, x):
        return self.main(x).view(-1, 1, 28, 28) # 생성자의 최종 반환값 : (batch_size x 1 x 28 x 28)

In [52]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.n_input = 784

        self.main = nn.Sequential(
            nn.Linear(self.n_input, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )


    def forward(self, x):
        x = x.view(-1, 784)
        return self.main(x) # 진짜 이미지일 확률 반환

In [53]:
generator = Generator(nz).to(device)
discriminator = Discriminator().to(device)

In [54]:
generator

Generator(
  (main): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): LeakyReLU(negative_slope=0.2)
    (6): Linear(in_features=1024, out_features=784, bias=True)
    (7): Tanh()
  )
)

In [55]:
discriminator

Discriminator(
  (main): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=256, out_features=1, bias=True)
    (10): Sigmoid()
  )
)

#### (5) 파라미터 설정

In [56]:
optim_generator = optim.Adam(generator.parameters(), lr=0.0002)
optim_discriminator = optim.Adam(discriminator.parameters(), lr=0.0002)

criterion = nn.BCELoss()

losses_generator = [] # 생성자의 손실함수값
losses_discriminator = [] # 판별자의 손실함수값
images = [] # 생성되는 이미지

In [57]:
def save_generator_image(image, path):
    save_image(image, path)

#### (6) 모델 학습

In [58]:
def train_discriminator(optimizer, data_real, data_fake):
    b_size = data_real.size(0) # batch_size 정보

    real_label = torch.ones(b_size, 1).to(device)
    fake_label = torch.zeros(b_size, 1).to(device)

    optimizer.zero_grad()

    output_real = discriminator(data_real)
    loss_real = criterion(output_real, real_label)

    output_fake = discriminator(data_fake)
    loss_fake = criterion(output_fake, fake_label)

    loss_real.backward()
    loss_fake.backward()

    optimizer.step()

    return loss_real + loss_fake

In [59]:
def train_generator(optimizer, data_fake):
    b_size = data_fake.size(0)

    real_label = torch.ones(b_size, 1).to(device)

    optimizer.zero_grad()

    output = discriminator(data_fake)
    loss = criterion(output, real_label)

    loss.backward()

    optimizer.step()

    return loss

In [ ]:
os.mkdir('/img')

In [61]:
generator.train()
discriminator.train()

for epoch in range(epochs):
    loss_generator = 0.0
    loss_discriminator = 0.0

    for idx, data in tqdm(enumerate(train_loader), total=int(len(train_dataset)/train_loader.batch_size)):
        image, _ = data
        image = image.to(device)
        b_size = len(image)

        for step in range(k):
            data_fake = generator(torch.randn(b_size, nz).to(device)).detach()
            data_real = image
            loss_discriminator += train_discriminator(optim_generator, data_real, data_fake)

        data_fake = generator(torch.randn(b_size, nz).to(device))
        loss_generator += train_generator(optim_generator, data_fake)

    generated_img = generator(torch.randn(b_size, nz).to(device)).cpu().detach()
    generate_img = make_grid(generated_img)

    save_generator_image(generated_img, f"/img/gen_img{epoch}.png")
    images.append(generated_img)

    epoch_loss_generator = loss_generator / idx
    epoch_loss_discriminator = loss_discriminator / idx

    losses_generator.append(epoch_loss_generator)
    losses_discriminator.append(epoch_loss_discriminator)

    print(f"Epoch {epoch} of {epochs}")
    print(f"Generator Loss: {epoch_loss_generator:.8f}, Discriminator Loss: {epoch_loss_discriminator:.8f}")

118it [00:12,  9.37it/s]                         


Epoch 0 of 20
Generator Loss: 0.42572817, Discriminator Loss: 1.78308558


118it [00:12,  9.32it/s]


Epoch 1 of 20
Generator Loss: 0.38395318, Discriminator Loss: 1.85356152


118it [00:12,  9.37it/s]                         


...

Epoch 17 of 20
Generator Loss: 0.37246937, Discriminator Loss: 1.87965000



118it [00:11, 10.16it/s]                         

Epoch 18 of 20
Generator Loss: 0.37249854, Discriminator Loss: 1.88050044



118it [00:11, 10.16it/s]                         

Epoch 19 of 20
Generator Loss: 0.37224826, Discriminator Loss: 1.88008535


#### (7) 생성자와 판별자의 오차 확인

- 생성자의 오차가 감소하면 판별자의 오차는 증가함

    - 생성자 : 점점 진짜와 같은 가짜 이미지를 만들어냄

    - 판별자 : 점점 가짜 이미지를 진짜라고 잘못 분류함

In [ ]:
plt.figure()

losses_generator = [f1.item() for f1 in losses_generator]
plt.plot(losses_generator, label='Generator Loss')

losses_discriminator = [f2.item() for f2 in losses_discriminator]
plt.plot(losses_discriminator, label='Discriminator Loss')

plt.legend()

#### (8) 생성된 이미지 확인

In [ ]:
fake_images = generator(torch.randn(b_size, nz).to(device))
for i in range(10):
    fake_images_img = np.reshape(fake_images.data.cpu().numpy()[i], (28, 28))
    plt.imshow(fake_images_img, cmap='grey')
    plt.save('/img/fake_images_img' + str(i) + '.png')
    plt.show()

---

## GAN 파생 기술

- GAN은 생성자와 판별자 중 한쪽으로 치우친 학습이 발생하면, 성능에 문제가 생길 수 있음
  
  -> 정상적인 분류 불가

---

#### DCGAN

- GAN 학습에 `CNN`을 사용한 모델

- Pooling을 모두 없애고, `Strided Convolution` 연산을 사용함

- Batch Normalization을 통해 안정적으로 gradient를 계산하도록 함

##### - Strided Convolution

- `판별자` 네트워크에서 사용

  - 이미지 특성을 추출하는 연산이 필요하기 때문

- 합성곱 연산에서 **1 이상의 정수 값** stride를 부여함

- 출력 feature map의 크기를 **줄이는 것**

<img src="https://thebook.io/img/080289/726_1.jpg" width="500px">

##### - Fractional-Strided Convolution

- `생성자` 네트워크에서 사용

  - 노이즈를 입력받아 실제 이미지와 같은 해상도로 크기를 키워야 하기 때문

- 합성곱 연산에서 **1보다 작은 분수 값** stride를 부여함

- 출력 feature map의 크기를 **키우는 것**

- ex) 2x2 feature map

  - 4개의 각 픽셀 주위에 `zero-padding`을 넣어줌

  - 그렇게 부풀려진 feature map에 Convolution 연산을 수행하여, 더 큰 feature map을 얻음

<img src="https://thebook.io/img/080289/726_2.jpg" width="500px">

---

#### cGAN

- GAN의 출력에 `조건`을 주어 이미지 생성을 통제할 수 있도록 하는 모델

  - 생성자에 노이즈 벡터와 조건 벡터 C를 함께 입력함
  
  - 판별자에도 마찬가지로 C가 추가됨
    
    -> 기존 이미지에서 `변형된 이미지` 생성이 가능해짐

<img src="https://thebook.io/img/080289/727.jpg" width="400px">

---

#### CycleGAN

- 이미지가 주어졌을 때 다른 이미지로 변형시키는 모델

- GAN, DCGAN은 랜덤 노이즈를 입력으로 하여 무작위 데이터가 생성되므로, 원하는 결과를 얻기 어려움
  
  -> PIX2PIX로 해결

##### - PIX2PIX

- 임의의 노이즈 벡터가 아닌 `이미지`를 입력 받음

- 다른 스타일의 이미지를 출력하는 **지도 학습** 알고리즘

- 생성자는 **진짜 같은 가짜 이미지**를 만들어야 하며, 또한 **정답 이미지와 같아야** 함

<img src="https://blog.kakaocdn.net/dn/dG3yWO/btq5i2yK9KW/BE1961uawwcYpfzCA9vzu1/img.png" width="450px">

- `데이터 쌍`이 필요하다는 단점 존재
    
    -> 쌍을 이루는 이미지 데이터를 얻기 어려움

<img src="https://www.codespeedy.com/wp-content/uploads/2020/07/pix2pix2.png" width="250px">

##### - CycleGAN

- `두 개의 생성자`(G, F)를 가짐

  - G : 이미지 X를 이미지 Y로 변환함

  - F : 이미지 Y를 다시 이미지 X로 변환함

- 판별자

    - Dx : 이미지 X를 위한 판별자
  
    - Dy : 이미지 Y를 위한 판별자

<img src="https://thebook.io/img/080289/731_1.jpg" width="400px">

- ex) 조랑말 이미지를 얼룩말 이미지로 변환

  - 생성자 G : 조랑말 `X` -> 얼룩말 `Y^`
  
  - 생성자 F : 얼룩말 `Y^` -> 조랑말 `X^`

    -> 원본 이미지 `X`가 생성된 이미지 `X^`와 얼마나 가까운 지 손실함수를 사용하여 계산함

- 순환 일관성 : `X` -> `Y^`, `Y^` -> `X^` 로 연결되는 것

- 정방향 일관성 : `X` -> `Y^`

- 역방향 일관성 : `Y^` -> `X^`

<img src="https://thebook.io/img/080289/731_2.jpg" width="500px">